In [1]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html

import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_style('whitegrid')
#%matplotlib inline

from pandas_datareader import data
from datetime import datetime

In [12]:
def WACC_func (sticker):
    # WACC Calculator for a given stock
    
    # Cost of Debt Calculation not tax exempted
    
    # Read longTermDebt
    dframe_Balance_Sheet = si.get_balance_sheet(sticker)
    longTermDebt_col = dframe_Balance_Sheet.loc['longTermDebt',:]
    longTermDebt = longTermDebt_col[0] / 1000000 #convert to millions
    shortTermDebt = 0
    
    # Read interestExpense, incomeTaxExpense, incomeBeforeTax
    dframe_Income_Statement = si.get_income_statement(sticker)
    interestExpense_col = dframe_Income_Statement.loc['interestExpense',:]
    interestExpense = abs(interestExpense_col[0]) / 1000000 #convert to millions
    incomeTaxExpense_col = dframe_Income_Statement.loc['incomeTaxExpense', :]
    incomeTaxExpense = abs(incomeTaxExpense_col[0]) / 1000000 #convert to millions
    incomeBeforeTax_col = dframe_Income_Statement.loc['incomeBeforeTax', :]
    incomeBeforeTax = abs(incomeBeforeTax_col[0]) / 1000000 #convert to millions

    CostOfDebt_PreTax = interestExpense/longTermDebt
    
    # Effective Tax Rate Calculation
    EffTaxRate = incomeTaxExpense/incomeBeforeTax
    
    # Cost of Debt * (1-T) Tax Exmpted
    CostOfDebt = CostOfDebt_PreTax * (1-EffTaxRate)
    
    # Cost of Equity Calculation
    
    # Read Risk Free Interest Rate - Treasury Yield 10 Years
    url_bond = 'https://finance.yahoo.com/bonds'
    dframe_list_Bond = pd.read_html(url_bond)
    TreasRate = dframe_list_Bond[0]
    RiskFreeRate = TreasRate.loc[2,'Last Price']/100
    
    # Read Beta
    dframe_Stats = si.get_stats(sticker)
    BetaVal = float(dframe_Stats.loc[0,'Value']) #data is in str, convert to float
    
    #MarketReturn = 8.5%
    MarketReturn = 0.085
    
    CostOfEquity = RiskFreeRate + (BetaVal * (MarketReturn - RiskFreeRate))
    
    # Weight of Debt and Equity Calculation
    
    TotalDebt = longTermDebt + shortTermDebt
    
    # Read Market Cap
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    MarketCap_tab = dframe_list_Summary[1]
    MarketCap_str = MarketCap_tab.loc[0,1]
    if MarketCap_str[-1:]=='B':    #check if Billion
        MarketCap = float(MarketCap_str[:-1])*1000
    if MarketCap_str[-1:]=='T':    #check if Trillion
        MarketCap = float(MarketCap_str[:-1])*1000*1000
    
    # Weight of Debt
    WeightOfDebt = TotalDebt / (TotalDebt + MarketCap)
    WeightOfEquity = MarketCap / (TotalDebt + MarketCap)
    
    # Weighted Average Cost of Capital (WACC) Calculation
    
    WACC = (CostOfDebt * WeightOfDebt) + (CostOfEquity * WeightOfEquity)
    
    return WACC

In [2]:
def stock_monte_carlo_func (start_price,days,mu,sigma):
    dt=1/days
    price = np.zeros(days)
    price[0] = start_price
    
    shock = np.zeros(days)
    drift = np.zeros(days)
    
    for x in range(1,days):
        shock[x] = np.random.normal(loc=mu*dt,scale=sigma*np.sqrt(dt))
        drift[x] = mu*dt
        price[x] = price[x-1] + (price[x-1] * (drift[x] + shock[x]))
        
    return price

In [3]:
def RiskVal_func (sticker_list, period):
    end = datetime.now()
    start = datetime(end.year-period, end.month, end.day)

    for stock in sticker_list:
        globals()[stock] = data.DataReader(stock,'yahoo',start,end)

    closing_df = data.DataReader(sticker_list,'yahoo',start,end)['Adj Close']
    tech_rets = closing_df.pct_change()
    rets = tech_rets.dropna()
    risk = 100*rets.quantile([0.05,0.01])

    risk_df = pd.DataFrame(risk.T)
    risk_df.index.names = ['Sticker']
    risk_df.rename({0.05:str(period)+'yr/95% WC Risk%', 0.01:str(period)+'yr/99% WC Risk%'}, axis=1, inplace=True)
    return risk_df

In [4]:
def GrahamInVal_func (sticker):
    #sticker = "CRM"   
    
    # Read EPS
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    Summary_tab = dframe_list_Summary[1]
    EPS_str = Summary_tab.loc[3,1]
    EPS = float(EPS_str)

    # Read P/E
    #PERatio_str = Summary_tab.loc[2,1]
    #PERatio_read = float(PERatio_str)
    PERatio_fixed = 7.0
    PERatio = PERatio_fixed

    # Read 5 Yr Growth Rate
    url_Analysis = 'https://finance.yahoo.com/quote/' + sticker + '/analysis?p=' + sticker
    dframe_list_Analysis = pd.read_html(url_Analysis)
    GrowthEst_tab = dframe_list_Analysis[5]
    GrowthEst_str = GrowthEst_tab.loc[4,sticker]
    GrowthEst = float(GrowthEst_str[:-1])
    GrowthScale = 1

    # Read AAACorpBond Yield
    url_AAACorpBond = 'https://ycharts.com/indicators/moodys_seasoned_aaa_corporate_bond_yield'
    dframe_list_AAACorpBond = pd.read_html(url_AAACorpBond)
    dframe_AAACorpBond = dframe_list_AAACorpBond[0]
    AAACorpBond_str = dframe_AAACorpBond.loc[0,'Unnamed: 1']
    AAACorpBond = float(AAACorpBond_str[:-1])
    CorpBond = 4.4

    GrahamInVal = round((EPS * (PERatio + (GrowthEst * GrowthScale)) * CorpBond) / AAACorpBond,2)

    # Read Current Val
    Summary_tab = dframe_list_Summary[0]
    PrevClose_str = Summary_tab.loc[0,1]
    PrevClose = round(float(PrevClose_str),2)

    if PrevClose < GrahamInVal:
        Decision = 'BUY'
    else:
        Decision = 'SELL'

    MarginRaw = (PrevClose/GrahamInVal)-1
    Margin = round(MarginRaw*100,2)
    #Margin = '{percent:.2%}'.format(percent=MarginRaw)

    return {'Sticker':sticker,
             'GrahamInVal($)':GrahamInVal,
             'PrevClose($)':PrevClose,
             'Decision':Decision,
             'Margin(%)':Margin,
             'EPS':EPS,
             'GrowthEst(%)':GrowthEst}